In [2]:
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mm import *

In [3]:
def load_parameter(filepath, evaluate=False):
    with open(filepath, 'r') as file:
        data = json.load(file)

    if evaluate:
        for parameter_list in [data["train_parameters"], data["test_parameters"]]:
            for parameter_set in parameter_list:
                parameter_set["radar_polangle"] = [[eval(angle) for angle in pair] for pair in parameter_set["radar_polangle"]]
                parameter_set["radar_altitude"] = [eval(value) for value in parameter_set["radar_altitude"]]
                parameter_set["warhead_v"] = [eval(value) for value in parameter_set["warhead_v"]]
                parameter_set["warhead_r"] = [eval(value) for value in parameter_set["warhead_r"]]
                parameter_set["track_interval"] = [eval(value) for value in parameter_set["track_interval"]]

    return data

In [4]:
# Load parameters.
parameters = load_parameter('parameters.json', evaluate=True)
parameters['train_parameters'][0]
# parameters['test_parameters'][0]

{'radar_altitude': [0, 15],
 'radar_polangle': [[0.7853981633974483, 0],
  [0.5235987755982988, 0.2617993877991494]],
 'warhead_v': [0, 7120.0, 1000.0],
 'warhead_r': [6771000.0, 0, 0],
 'track_interval': [60, 120]}

In [6]:
def display_track(initial_parameters_list, combine = True):
    R = 6371e3
    if combine:
        fig = plt.figure(figsize=(10, 7))
        ax = fig.add_subplot(111, projection='3d')
        for parameters in initial_parameters_list:
            a, e, i, Omega, omega, M0 = orbital_elements(parameters['warhead_r'], parameters['warhead_v'] , False)
            orbit_points, _ = calculate_orbit_points(a, e, i, Omega, omega, M0, 1000)
            view_orbit(ax, parameters['warhead_r'], parameters['warhead_v'], None, 0, 45, 'Warhead Orbit', orbit_points = orbit_points)
            ax.scatter(orbit_points[-1][0], orbit_points[-1][1], orbit_points[-1][2], marker='x', color = 'k')
            ax.scatter(orbit_points[0][0], orbit_points[0][1], orbit_points[0][2], marker='.', color = 'k')
            start_point = orbit_points[parameters['track_interval'][0]]
            end_point = orbit_points[parameters['track_interval'][1]]

            for idx,radar_pa in enumerate(parameters['radar_polangle']):
                r = R + parameters['radar_altitude'][idx]
                radar = np.array([r * np.cos(radar_pa[0]) * np.cos(radar_pa[1]), 
                                    r * np.sin(radar_pa[0]) * np.cos(radar_pa[1]), 
                                    r * np.sin(radar_pa[1])]) 
                ax.scatter(start_point[0], start_point[1], start_point[2], color = 'b', s=5)
                ax.scatter(end_point[0], end_point[1], end_point[2] , color = 'b', s=5)
                
                ax.scatter(radar[0], radar[1], radar[2] , color = 'r', s=5)
                
                vertices = [ start_point, end_point, radar]
                poly3d = [vertices]

                ax.add_collection3d(Poly3DCollection(poly3d, facecolors='gray', linewidths=0.5, edgecolors='b', alpha=0.5))
                
    
    else:
        for num,parameters in enumerate(initial_parameters_list):
            fig = plt.figure(figsize=(10, 10))
            ax = fig.add_subplot(111, projection='3d')
            a, e, i, Omega, omega, M0 = orbital_elements(parameters['warhead_r'], parameters['warhead_v'] , False)
            orbit_points, _ = calculate_orbit_points(a, e, i, Omega, omega, M0, 1000)
            view_orbit(ax, parameters['warhead_r'], parameters['warhead_v'], None, 0, 45, None, orbit_points = orbit_points)
            ax.scatter(orbit_points[-1][0], orbit_points[-1][1], orbit_points[-1][2], marker='x', color = 'k')
            ax.scatter(orbit_points[0][0], orbit_points[0][1], orbit_points[0][2], marker='.', color = 'k')
            start_point = orbit_points[parameters['track_interval'][0]]
            end_point = orbit_points[parameters['track_interval'][1]]

            for idx,radar_pa in enumerate(parameters['radar_polangle']):
                r = R + parameters['radar_altitude'][idx]
                radar = np.array([r * np.cos(radar_pa[0]) * np.cos(radar_pa[1]), 
                                    r * np.sin(radar_pa[0]) * np.cos(radar_pa[1]), 
                                    r * np.sin(radar_pa[1])]) 
                ax.scatter(start_point[0], start_point[1], start_point[2], color = 'b', s=5)
                ax.scatter(end_point[0], end_point[1], end_point[2] , color = 'b', s=5)
                
                ax.scatter(radar[0], radar[1], radar[2] , color = 'r', s=5)
                
                # vertices = [radar, start_point, end_point]
                # poly3d = [vertices]

                # ax.add_collection3d(Poly3DCollection(poly3d, facecolors='gray', linewidths=0.5, edgecolors='b', alpha=0.5))
                verts = orbit_points[parameters['track_interval'][0]:parameters['track_interval'][1]].tolist()
                verts.append(radar.tolist())
                ax.add_collection3d(Poly3DCollection([verts], facecolors='gray', linewidths=0.5, edgecolors='b', alpha=0.5))

                ax.set_axis_off()
            
            fig.savefig(f'./{num}.png', format='png', dpi=500, bbox_inches='tight') # change it.
            plt.close(fig)

In [8]:
%matplotlib inline
display_track(parameters['train_parameters'], combine = False)